In [1]:
#import cv2
import numpy as np
from scipy import ndimage

#from sa_decomp_layer import SADecompLayer

import os
#os.environ["CUDA_VISIBLE_DEVICES"]="-1"  #disables GPU 
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
#tf.__version__
from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())

plt.rcParams['figure.figsize'] = [10,10]







In [2]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import decode_predictions
from tensorflow.keras.applications import VGG16

model = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(224, 224, 3))

#model.summary()

In [3]:
def get_filter(layer):
    layer = model.layers[layer]

    # check for convolutional layer
    if 'conv' not in layer.name:
        return None
    # get filter weights
    filters, biases = layer.get_weights()
    return (filters)
    #print(layer.name, filters.shape)

# Dominant orientations for Block 1

In [4]:
from skimage.filters import sobel_h
from skimage.filters import sobel_v

def getSobelAngle(f):

    s_h = sobel_h(f)
    s_v = sobel_v(f)

    return(np.arctan2(s_h,s_v)+(np.pi/2))

def getSymAntiSym(filter):

    #patches = extract_image_patches(filters, [1, k, k, 1],  [1, k, k, 1], rates = [1,1,1,1] , padding = 'VALID')
    #print(patches)
    mat_flip_x = np.fliplr(filter)

    mat_flip_y = np.flipud(filter)

    mat_flip_xy =  np.fliplr( np.flipud(filter))

    sum = filter + mat_flip_x + mat_flip_y + mat_flip_xy
    mat_sum_rot_90 = np.rot90(sum)
    
    return  (sum + mat_sum_rot_90) / 8, filter - ((sum + mat_sum_rot_90) / 8)


def normalizeToOne(mat):
    return mat/np.linalg.norm(mat)


def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))


In [5]:

matplotlib.use('TkAgg')

from mpl_toolkits import mplot3d
filters = get_filter(1)

fig = plt.figure()
ax = plt.axes(projection='3d')

for i in range(filters.shape[-1]):
    filter = filters[:,:,:, i]
    #for j in range(filter.shape[-1]):
    f = filter[:,:, 1]
    # Data for a three-dimensional line
    '''zline = np.linspace(0, 15, 1000)
    xline = np.sin(zline)
    yline = np.cos(zline)
    ax.plot3D(xline, yline, zline, 'gray')'''

    sym, anti = getSymAntiSym(normalizeToOne(f))
    sym_mag = np.linalg.norm(sym)
    anti_mag = np.linalg.norm(anti)

    theta = getSobelAngle(f)

    # Data for three-dimensional scattered points
    zdata = sym_mag
    xdata = anti_mag*np.sin(theta) 
    ydata = anti_mag*np.cos(theta) 
    ax.scatter3D(xdata, ydata, zdata, c='red', cmap='Greens')
plt.show()
matplotlib.pyplot.waitforbuttonpress();

: 